In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import time
import torch
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Instantiating the pretrained model 
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [3]:
def calculating_score(tweet):
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [4]:
#Loading the twitter data
covid_tweets = pd.read_csv('preprocessed_tweets.csv', index_col=[0])

In [5]:
# Looking at na values
print('Before dropping: {}'.format(covid_tweets['Text'].isna().sum()))

#dropping na values
covid_tweets =  covid_tweets[covid_tweets['Text'].notna()]

#Resetting index
covid_tweets = covid_tweets.reset_index(drop=True)

#Creating new column called sentiment
covid_tweets['Sentiment']=0

# Checking at na values after dropping
print('After dropping: {}'.format(covid_tweets['Text'].isna().sum()))

Before dropping: 64
After dropping: 0


In [6]:
#Calculating sentiment score for each tweet 
for i in tqdm(range(len(covid_tweets['Text']))):
    covid_tweets['Sentiment'][i]= calculating_score(covid_tweets['Text'][i])

100%|█████████████████████████████████████| 19940/19940 [34:06<00:00,  9.74it/s]


In [7]:
covid_tweets.to_csv('tweets_w_bert_sentiments.csv')